### 1. Non-dominated Sorting Genetic Algorithm (NSGA-II)
##### 2. Multi-Objective Particle Swarm Optimization (MOPSO)
##### 3. Multi-Objective Differential Evolution (MODE)
##### 4. Multi-Objective Genetic Programming (MOGP)

In [1]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from model.networks_bf import get_all_devices_combined
from model.services_bf import get_all_services
import pandas as pd

# Create an empty DataFrame to store the results
columns = ['config', 'physical_machines', 'services', 'generations', 'alpha', 'beta', 'latency', 'energy', 'placement_physical', 'placement_vm', 'is_perfect']
results_headers = pd.DataFrame(columns=columns)
results_headers.to_csv('results/results_mini_nsga.csv', mode='a', index=False)

results_df = pd.DataFrame()

for c in range(1, 101):
    
    print(f"Processing config-{c}....")
    
    physical_machines = get_all_devices_combined(f"config-{c}")
    services = get_all_services(f"config-{c}")
    
    # Define the problem-specific parameters
    n = len(physical_machines)  # Number of physical machines
    s = len(services)  # Number of services
    
    # Values for num_generations and alpha
    num_generations_values = [200]
    alpha_values = [0.5]
    
    # Outer loop: iterate through num_generations values
    for num_generations in num_generations_values:
        # Inner loop: iterate through alpha values
        for alpha in alpha_values:
            
            beta = round(1 - alpha, 1)   # Weight for energy consumption
            
            if hasattr(creator, 'FitnessMulti'):
                del creator.FitnessMulti
            if hasattr(creator, 'Individual'):
                del creator.Individual
            # Define the problem as a multi-objective optimization problem
            creator.create("FitnessMulti", base.Fitness, weights=[-1])
            creator.create("Individual", list, fitness=creator.FitnessMulti)
            
            # Define parameters. This will be loaded from the database.
            # Here, we use random values for demonstration purposes.
            # Replace these with actual data when implementing in a real system.
            
            # Variable number of virtual machines for each physical machine
            max_v = max(map(lambda machine: len(machine.guest_machines), physical_machines))
            
            ################ Physical and virtual machine configurations ##################
            
            # Initialization
            R = np.empty(shape=(n, max_v))  # Request network delay of the machine
            R.fill(999999) # Initialize with very large value
            S = np.empty(shape=(n, max_v))  # Response network delay of the machine
            S.fill(999999) # Initialize with very large value
            X = np.empty(shape=(n, max_v))  # Maximum IPS possible
            X.fill(1) # Initialize with 1
            
            PI = np.empty(shape=(n, max_v))  # Power consumption in idle
            PI.fill(999) # Initialize with very large value
            PM = np.empty(shape=(n, max_v))  # Maximum power consumption
            PM.fill(999) # Initialize with very large value
            
            # Fill in with the actual numbers
            for i in range(len(physical_machines)):
                for j in range(len(physical_machines[i].guest_machines)):
                    R[i][j] = physical_machines[i].guest_machines[j].net_delay_request
                    S[i][j] = physical_machines[i].guest_machines[j].net_delay_response
                    X[i][j] = physical_machines[i].guest_machines[j].max_instructions_per_second
                    PI[i][j] = physical_machines[i].guest_machines[j].idle_cpu_utilization
                    PM[i][j] = physical_machines[i].guest_machines[j].max_cpu_utilization
            
            ################ Service configurations ###################
            
            # Acceptable latencies for services
            SAL = [service.acceptable_latency for service in services]
            # Initialize IPS values for services (replace with actual data)
            SIPS = [service.average_instructions_per_second for service in services]
            # Layers considered for service
            SL = [service.layer for service in services]
            
            # Define the objective functions
            def evaluate_individual(individual):
                I = np.empty(shape=(n, max_v))  # IPS currently executed
                I.fill(0) # Initialize with 0
                # Calculate energy consumption based on the parameters
                E = PI + (PM - PI) * (I / X)
                total_latency = 0.0
                total_energy = 0.0
                
                is_perfect = [False] * s # To check if the permutation is perfect
                
                for p in range(len(physical_machines)):
                    for v in range(len(physical_machines[p].guest_machines)):
                        total_energy += E[p][v]
                
                success_count = 0
                vm_placements = []
                for i in range(s):
                    p = individual[i]  # Physical machine where service is placed
                    
                    # Find the virtual machine with the least objective functiopn value
                    least_objective = float("inf")
                    v_idx = 0
                    for v in range(len(physical_machines[p].guest_machines)):
                        latency = R[p][v] + (SIPS[i] * 1000 / X[p][v]) + S[p][v]
                        energy = PI[p][v] + (PM[p][v] - PI[p][v]) * (I[p][v] / X[p][v])
                        objective = alpha * latency + beta * energy
                        if objective < least_objective:
                            least_objective = objective
                            v_idx = v
                    
                    if p < n:
                        for v in range(len(physical_machines[p].guest_machines)):
                            vm = physical_machines[p].guest_machines[v]
                            latency = R[p][v] + (SIPS[i] * 1000 / X[p][v]) + S[p][v]
                            if SL[i] is not None and SL[i] != vm.layer: # Consider the layer constrains provided by the service
                                continue
                            if latency > SAL[i]:
                                continue
                            v_idx = v
                            is_perfect[i] = True
                            break
                    
                    total_latency += (R[p][v_idx] + (SIPS[i] * 1000 / X[p][v_idx]) + S[p][v_idx])
                    I[p][v_idx] = I[p][v_idx] + SIPS[i] # Update the current instructions per second
                    total_energy += PI[p][v_idx] + (PM[p][v_idx] - PI[p][v_idx]) * (I[p][v_idx] / X[p][v_idx]) # Calculate energy consumption
                    success_count += 1
                    vm_placements.append(v_idx)
                
                # if all the services are not placed for this particular individual,
                # return a large fitness value to ignore it
                if success_count is not len(services):
                    total_latency = float('inf')
                    total_energy = float('inf')

                individual.vms = vm_placements
                individual.is_perfect = is_perfect
                individual.total_latency = total_latency
                individual.total_energy = total_energy
                return [total_latency * alpha + total_energy * beta]
            
            # Create a DEAP toolbox and register functions
            toolbox = base.Toolbox()
            toolbox.register("attr_int", random.randint, 0, n - 1)
            toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_int,), n=s)
            toolbox.register("population", tools.initRepeat, list, toolbox.individual)
            
            # Evaluation function
            toolbox.register("evaluate", evaluate_individual)
            toolbox.register("mate", tools.cxTwoPoint)
            toolbox.register("mutate", tools.mutUniformInt, low=0, up=n - 1, indpb=0.2)
            toolbox.register("select", tools.selBest)
            
            # Crossover, mutation, and selection registration
    
            # Create the initial population
            population = toolbox.population(n=100)
        
            # Create statistics object to track performance
            stats = tools.Statistics(lambda ind: ind.fitness.values)
            stats.register("avg", np.mean, axis=0)
            stats.register("min", np.min, axis=0)
            stats.register("max", np.max, axis=0)
        
            # Create a logbook to log statistics
            logbook = tools.Logbook()
            logbook.header = "gen", "evals", "avg", "min", "max"
        
            # Run NSGA-II algorithm
            algorithms.eaMuPlusLambda(population, toolbox, mu=100, lambda_=200, cxpb=0.7, mutpb=0.2, ngen=num_generations, stats=stats,
                                halloffame=None, verbose=True)
        
            # Extract the Pareto front solutions (not necessarily Pareto-optimal in a basic GA)
            pareto_front = tools.sortNondominated(population, len(population), first_front_only=True)[0]
            
            # # Extract objective values from the Pareto front
            # solutions = [ind.fitness.values[0] * alpha + ind.fitness.values[1] * beta for ind in pareto_front]
            # min_index = solutions.index(min(solutions))
            
            # Get the fitness values and individual of the minimum index
            ind = pareto_front[0]
            
            # Append the data to the results DataFrame
            results_df = results_df._append({'config': f'config-{c}',
                                             'physical_machines': n,
                                             'services': s,
                                             'generations': num_generations,
                                             'alpha': alpha,
                                             'beta': beta,
                                             'latency': ind.total_latency,
                                             'energy': ind.total_energy,
                                             'placement_physical': ind,
                                             'placement_vm': ind.vms,
                                             'is_perfect': ind.is_perfect}, ignore_index=True)
            
    # Save the results DataFrame to a CSV file
    results_df.to_csv('results/results_mini_nsga.csv', mode='a', index=False, header=False)
    results_df = pd.DataFrame()

Processing config-1....
gen	nevals	avg             	min             	max             
0  	100   	[17640.05318536]	[12258.28567704]	[27956.95537123]
1  	179   	[14311.49240255]	[11985.3201098] 	[16105.50089847]
2  	185   	[12768.2096983] 	[10699.81439899]	[13753.01315475]
3  	179   	[11863.67532928]	[10636.95570545]	[12540.78947864]
4  	178   	[11148.1284424] 	[10404.17856285]	[11734.36856599]
5  	181   	[10736.04535151]	[10393.30243722]	[10962.75384845]
6  	180   	[10550.15462527]	[10347.01007023]	[10688.93827337]
7  	179   	[10441.14450658]	[10314.47068862]	[10515.92752258]
8  	176   	[10392.82101713]	[10314.47068862]	[10419.07297457]
9  	180   	[10375.67465381]	[10314.47068862]	[10404.17856285]
10 	184   	[10352.65588211]	[10314.47068862]	[10393.30243722]
11 	188   	[10332.62822785]	[10314.47068862]	[10347.01007023]
12 	177   	[10313.75829277]	[10278.85089584]	[10314.47068862]
13 	179   	[10311.49332134]	[10278.85089584]	[10314.47068862]
14 	177   	[10304.469993]  	[10278.85089584]	[